In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Load the WTI futures time series
# Ensure the file contains 'Timestamp' and 'Price' columns
data = pd.read_csv("wti_futures_data.csv", parse_dates=['Timestamp'])

# Sort the data by date and time if needed
data = data.sort_values('Timestamp').reset_index(drop=True)

# Calculate intraday volatility (standard deviation of returns) over 30-minute intervals
data.set_index('Timestamp', inplace=True)
data['Return'] = data['Price'].pct_change()  # Calculate returns

# Resample the time series into 30-minute intervals
volatility_30min = data['Return'].rolling('30min').std() * np.sqrt(30)  # Annualize if needed

# Detect anomalies - simple threshold based on a high standard deviation (e.g., 3 sigma)
threshold = volatility_30min.mean() + 3 * volatility_30min.std()  # Adjustable based on tolerance
anomalies = volatility_30min[volatility_30min > threshold]

# Visualize volatility and anomalies
plt.figure(figsize=(12, 6))
plt.plot(volatility_30min, label="30-Min Volatility")
plt.scatter(anomalies.index, anomalies, color='red', label="Volatility Spikes (Anomalies)", marker='o')
plt.xlabel("Time")
plt.ylabel("Volatility")
plt.title("30-Min Volatility with Identified Spikes")
plt.legend()
plt.show()
